In [ ]:
import numpy as np
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import math
import json
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle
import pandas as pd
import tensorflow as tf

In [ ]:
def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y


def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="validation accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="validation error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    #axs[1].set_title("Error eval")

    plt.show()

def build_model_lstm(input_shape):
    """Generates RNN-LSTM model
    :param input_shape (tuple): Shape of input set
    :return model: RNN-LSTM model
    """

    # build network topology
    model = keras.Sequential()

    # 2 LSTM layers
    model.add(keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True))


    #LSTM
    model.add(keras.layers.LSTM(64,  return_sequences=True))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.LSTM(64))
    model.add(keras.layers.Dropout(0.3))

    # dense layer
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.3))



    # output layer
    model.add(keras.layers.Dense(152, activation='softmax'))

    return model

def build_model_cnn(input_shape):

    # Let's design the model architecture.
    model = keras.models.Sequential([
        
        keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D((2,2), padding='same'),
        keras.layers.BatchNormalization(),

        keras.layers.Conv2D(64, (3,3), activation='relu'),
        keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        keras.layers.BatchNormalization(),

        keras.layers.Conv2D(32, (2,2), activation='relu'),
        keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'), 
        keras.layers.Dense(152, activation='softmax')
    ])

    return model
    

In [ ]:
mfcc_path = '../input/feijai-mfcc-3'


In [ ]:
X=[]
y=[]
map=[]
df=pd.DataFrame()
for dirpath, dirnames, filenames in os.walk(mfcc_path):
    for i, file in enumerate(filenames):
        mfcc = np.load(dirpath+'/'+file)
        label=file.split('.')[0]

        if i==0:
            X=mfcc
        else:
            X=np.concatenate((X,mfcc))
        
        tags=[i]*mfcc.shape[0]
        #print(mfcc.shape[0])
        y=np.concatenate((y, tags))
        labels=[label]*mfcc.shape[0]
        map=np.concatenate((map, labels))

y = [int(y) for y in y]
y = np.array(y)

In [ ]:
#CALL_BACK = EarlyStopping('val_loss',patience=3)
train_X, train_y = X, y

#shuffle避免validation的bias過大
train_X, train_y=shuffle(train_X, train_y,random_state=0)

"""
kf = StratifiedKFold(n_splits=5,                      
            random_state=10,
            shuffle=True)
            
kf.get_n_splits(train_X,train_y)
"""
T_train_acc = []
T_test_acc = []
T_train_loss = []
T_test_loss = []

#for train_index, test_index in kf.split(train_X,train_y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #X_train, X_test = train_X[train_index], train_X[test_index]
    #y_train, y_test = train_y[train_index], train_y[test_index]

X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2, random_state=10)   

# LSTM network
input_shape = (X_train.shape[1],X_train.shape[2]) # X, 13
model = build_model_lstm(input_shape)

"""
# CNN network
shape_1 = X_train.shape[1]
shape_2 = X_train.shape[2]

X_train = X_train.reshape(X_train.shape[0], shape_1, shape_2, 1)
X_test = X_test.reshape(X_test.shape[0], shape_1, shape_2, 1)

input_shape = (shape_1,shape_2,1)
model = build_model_cnn(input_shape)
"""
    

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
if (len(T_train_acc)) < 1:
    model.summary()



# train model


history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2) #, callbacks=[CALL_BACK]
T_train_acc.append(history.history['val_accuracy'][-1])
T_train_loss.append(history.history['val_loss'][-1])
print('\n\n5-fold cross',len(T_train_acc))
print('\nValidation accuracy:', history.history['val_accuracy'][-1])
print('\nValidation loss:', history.history['val_loss'][-1])

# plot accuracy/error for training and validation
plot_history(history)

# evaluate model on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
T_test_loss.append(test_loss)
T_test_acc.append(test_acc)
print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

print('\n\nMean of validation accuracy:')
print(np.mean(T_train_acc))
print('Mean of validation loss:')
print(np.mean(T_train_loss))
print('Mean of test accuracy:')
print(np.mean(T_test_acc))
print('Mean of test loss:')
print(np.mean(T_test_loss))

In [ ]:
model.save('../BS_LSTM_100epoch_3.h5')